In [43]:
# Range dependent Hamiltonian Algorithm for numerical QUBO formulation

# This code only use one subdomain because of time limit of D-Wave account
# For a genearal code, please use additional "for" loop in "cef2"

# 4 by 4 matrix A
# Range of x_i = (-128, 127)
# Subrange of x_i = (0, 3) = q_(i,1) + 2q(i,2)

import numpy as np
import random, math
import copy
import sys
import math

Dimension = 4
A = np.random.randint(-10, 10, size=(Dimension,Dimension))
np.set_printoptions(threshold=sys.maxsize)
print("Matrix A:")
print(A)

x_fin = np.random.randint(-128, 127, size=Dimension)
print("\nSolution vector x:")
print(x_fin)

b = np.dot(A, x_fin)
print("\nVector b:")
print(b)

Matrix A:
[[ 3  8  1  2]
 [ 3 -2  5  4]
 [ 4 -3  7  0]
 [ 4  2 -2  0]]

Solution vector x:
[ 52 -87  99  88]

Vectpr b:
[-265 1177 1162 -164]


In [44]:
qubits = 2
# Calculate translation number including solution vector x
Trs = np.zeros(Dimension)
x_qubit = np.zeros(Dimension)
sub_size = pow(2,qubits)
for k in range(Dimension):
    Val = divmod(x_fin[k], sub_size)
    Trs[k] = sub_size*Val[0]
    x_qubit[k] = Val[1]
print("Translation vector T:")
print(Trs)
print("\nQubit notation:")
print(x_qubit)
c = b - np.dot(A, Trs)
print("\nUpdated vector c:")
print(c)

Translation vector T:
[ 52. -88.  96.  88.]

Qubit notation:
[0. 1. 3. 0.]

Updated vector c:
[11. 13. 18. -4.]


In [48]:
QM = np.zeros((qubits*Dimension, qubits*Dimension))
### Linear terms ###
for k in range(Dimension):
    for i in range(Dimension):
        for l in range(qubits):
            cef1 = pow(2,2*l)*pow(A[k][i],2)
            cef2 = pow(2,l+1)*A[k][i]*c[k]
            po = qubits*i + l
            QM[po][po] = QM[po][po] + cef1 - cef2
 
### First quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension):
        for l1 in range(qubits-1):
            for l2 in range(l1+1,qubits):
                qcef = pow(2, l1+l2+1)*pow(A[k][i],2)
                po1 = qubits*i + l1
                po2 = qubits*i + l2
                QM[po1][po2] = QM[po1][po2] + qcef
 
### Second quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension-1):
        for j in range(i+1,Dimension):
            for l1 in range(qubits):
                for l2 in range(qubits):  
                    qcef = pow(2, l1+l2+1)*A[k][i]*A[k][j] 
                    po1 = qubits*i + l1
                    po2 = qubits*j + l2
                    QM[po1][po2] = QM[po1][po2] + qcef

# Print Matrix Q
print("# Matrix Q is")
print(QM)
print("\nMinimum energy is ",-np.dot(c,c))
print("\n")

# Print Python code for the run in D-Wave quantum processing unit
print("Running code for D-Wave:\n")
print("from dwave.system import DWaveSampler, EmbeddingComposite")
print("sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))\n")
print("linear = {", end = "")
for i in range(qubits*Dimension-1): 
    linear = i + 1
    print ("('q",linear,"','q",linear,"'):",format(QM[i][i]),sep='', end = ", ")
print ("('q",qubits*Dimension,"','q",qubits*Dimension,"'):",format(QM[qubits*Dimension-1][qubits*Dimension-1]),"}", sep='')

print("\nquadratic = {", end = "")
for i in range(qubits*Dimension-1):
    for j in range(i+1,qubits*Dimension):
        if QM[i][j] != 0:
            qdrt1 = i + 1
            qdrt2 = j + 1
            if i == qubits*Dimension-2 and j == qubits*Dimension-1:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), "}", sep='')
            else:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")

print("\nQ = dict(linear)")
print("Q.update(quadratic)\n")

qa_iter = 1000
print("sampleset = sampler_auto.sample_qubo(Q, num_reads=",qa_iter,")", sep = "")
print("print(sampleset)")      

# Matrix Q is
[[-206.  200.   28.   56.   76.  152.   36.   72.]
 [   0. -312.   56.  112.  152.  304.   72.  144.]
 [   0.    0.   81.  324.  -54. -108.   16.   32.]
 [   0.    0.    0.  324. -108. -216.   32.   64.]
 [   0.    0.    0.    0. -341.  316.   44.   88.]
 [   0.    0.    0.    0.    0. -524.   88.  176.]
 [   0.    0.    0.    0.    0.    0. -128.   80.]
 [   0.    0.    0.    0.    0.    0.    0. -216.]]

Minimum energy is  -630.0


Running code for D-Wave:

from dwave.system import DWaveSampler, EmbeddingComposite
sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))

linear = {('q1','q1'):-206.0, ('q2','q2'):-312.0, ('q3','q3'):81.0, ('q4','q4'):324.0, ('q5','q5'):-341.0, ('q6','q6'):-524.0, ('q7','q7'):-128.0, ('q8','q8'):-216.0}

quadratic = {('q1','q2'):200.0, ('q1','q3'):28.0, ('q1','q4'):56.0, ('q1','q5'):76.0, ('q1','q6'):152.0, ('q1','q7'):36.0, ('q1','q8'):72.0, ('q2','q3'):56.0, ('q2','q4'):112.0, ('q2','q5'):152.0, ('q2','q6'):304.0, ('q2','q7

In [47]:
from dwave.system import DWaveSampler, EmbeddingComposite
sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))

linear = {('q1','q1'):-206.0, ('q2','q2'):-312.0, ('q3','q3'):81.0, ('q4','q4'):324.0, ('q5','q5'):-341.0, ('q6','q6'):-524.0, ('q7','q7'):-128.0, ('q8','q8'):-216.0}

quadratic = {('q1','q2'):200.0, ('q1','q3'):28.0, ('q1','q4'):56.0, ('q1','q5'):76.0, ('q1','q6'):152.0, ('q1','q7'):36.0, ('q1','q8'):72.0, ('q2','q3'):56.0, ('q2','q4'):112.0, ('q2','q5'):152.0, ('q2','q6'):304.0, ('q2','q7'):72.0, ('q2','q8'):144.0, ('q3','q4'):324.0, ('q3','q5'):-54.0, ('q3','q6'):-108.0, ('q3','q7'):16.0, ('q3','q8'):32.0, ('q4','q5'):-108.0, ('q4','q6'):-216.0, ('q4','q7'):32.0, ('q4','q8'):64.0, ('q5','q6'):316.0, ('q5','q7'):44.0, ('q5','q8'):88.0, ('q6','q7'):88.0, ('q6','q8'):176.0, ('q7','q8'):80.0}

Q = dict(linear)
Q.update(quadratic)

sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)

   q1 q2 q3 q4 q5 q6 q7 q8 energy num_oc. chain_.
0   0  0  1  0  1  1  0  0 -630.0     543     0.0
17  0  0  1  0  1  1  0  0 -630.0       1   0.125
37  0  0  1  0  1  1  0  0 -630.0       1   0.125
1   0  0  1  0  1  1  1  0 -610.0     187     0.0
2   1  0  0  0  0  1  1  0 -582.0      22     0.0
3   1  0  1  0  1  1  0  0 -580.0      14     0.0
4   1  0  0  0  0  1  0  0 -578.0      16     0.0
5   1  0  1  0  0  1  0  0 -577.0      20     0.0
6   0  0  1  0  0  1  1  0 -575.0      46     0.0
7   1  0  1  0  0  1  1  0 -565.0      18     0.0
36  1  0  1  0  0  1  1  0 -565.0       1   0.125
8   0  0  0  0  0  1  1  0 -564.0       6     0.0
9   0  0  0  0  0  1  0  1 -564.0       7     0.0
10  0  0  1  0  0  1  0  1 -559.0      11     0.0
11  0  0  1  0  0  1  0  0 -551.0      10     0.0
12  0  0  1  0  1  1  0  1 -550.0      11     0.0
13  0  0  0  1  1  1  0  0 -549.0      32     0.0
14  0  0  0  0  1  1  0  0 -549.0       8     0.0
15  1  0  0  0  0  1  0  1 -546.0       5     0.0


In [ ]:
Minimum energy is  -162165.0